In [1]:
import importlib
from component import FeatureSelection
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
context = InteractiveContext()

In [3]:
import urllib.request
import tempfile
import os

# getting data and setup CsvExampleGen
DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-data_4rszyag/data.csv',
 <http.client.HTTPMessage at 0x7fba4737b150>)

In [4]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-08-19T05_49_05.332863-c2l3rf6g/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:25648,xor_checksum:1629332345,sum_checksum:1629332345"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
# give path to the module file
feature_selector = FeatureSelection(orig_examples = example_gen.outputs['examples'],
                                   module_file="module_file")

In [6]:
example_gen.outputs['examples'].get()[0].uri

'/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-08-19T05_49_05.332863-c2l3rf6g/CsvExampleGen/examples/1'

In [7]:
context.run(feature_selector)

2021-08-19 05:49:07.008928: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-19 05:49:07.110435: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


ExecutionResult(
    component_id: FeatureSelection
    execution_id: 2
    outputs:
        feature_selection: Channel(
            type_name: Feature Selection
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-08-19T05_49_05.332863-c2l3rf6g/FeatureSelection/feature_selection/2"
        properties {
          key: "p_values"
          value {
            struct_value {
              fields {
                key: "body_mass_g"
                value {
                  number_value: 4.195541966407283e-05
                }
              }
              fields {
                key: "culmen_depth_mm"
                value {
                  number_value: 8.747194971213637e-05
                }
              }
              fields {
                key: "culmen_length_mm"
                value {
                  number_value: 0.0007791004116491523
                }
              }
              fields {
                key: "flipper_length_mm"
                value {
                  number_value: 3.104844357027674e-05
                }
              }
            }
          }
        }
        properties {
          key: "scores"
          value {
            struct_value {
              fields {
                key: "body_mass_g"
                value {
                  number_value: 20.15780587994667
                }
              }
              fields {
                key: "culmen_depth_mm"
                value {
                  number_value: 18.68838478142841
                }
              }
              fields {
                key: "culmen_length_mm"
                value {
                  number_value: 14.314741244525251
                }
              }
              fields {
                key: "flipper_length_mm"
                value {
                  number_value: 20.7599237548046
                }
              }
            }
          }
        }
        properties {
          key: "selected_data"
          value {
            struct_value {
              fields {
                key: "__value__"
                value {
                  list_value {
                    values {
                      list_value {
                        values {
                          number_value: 0.2916666567325592
                        }
                        values {
                          number_value: 0.1525423675775528
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 0.3055555522441864
                        }
                        values {
                          number_value: 0.23728813230991364
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 0.1527777761220932
                        }
                        values {
                          number_value: 0.38983049988746643
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 0.2083333283662796
                        }
                        values {
                          number_value: 0.35593220591545105
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 0.2638888955116272
                        }
                        values {
                          number_value: 0.3050847351551056
                        }
                      }
                    }
                    values {
                   